In [1]:
!pip install numpy==1.16.1

  Attempting uninstall: numpy
    Found existing installation: numpy 1.16.5
    Uninstalling numpy-1.16.5:


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
ERROR: umap-learn 0.4.2 has requirement numba!=0.47,>=0.46, but you'll have numba 0.45.1 which is incompatible.
ERROR: umap-learn 0.4.2 has requirement numpy>=1.17, but you'll have numpy 1.16.1 which is incompatible.
ERROR: phik 0.9.11 has requirement joblib>=0.14.1, but you'll have joblib 0.13.2 which is incompatible.
ERROR: mlxtend 0.17.2 has requirement numpy>=1.16.2, but you'll have numpy 1.16.1 which is incompatible.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\numpy\\compat\\py3k.py'
Consider using the `--user` option or check the permissions.



In [4]:
import numpy
from numpy import array
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import load_model
import re
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
# fix random seed for reproducibility
numpy.random.seed(7)

In [11]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=10000)
'''Inspect a sample review and its label.Note that the review is stored as a sequence of integers. These are word IDs that 
have been pre-assigned to individual words, and the label is an integer (0 for negative, 1 for positive).'''
print('---review---')
print(X_train[6])
print('---label---')
print(y_train[6])

---review---
[1, 6740, 365, 1234, 5, 1156, 354, 11, 14, 5327, 6638, 7, 1016, 2, 5940, 356, 44, 4, 1349, 500, 746, 5, 200, 4, 4132, 11, 2, 9363, 1117, 1831, 7485, 5, 4831, 26, 6, 2, 4183, 17, 369, 37, 215, 1345, 143, 2, 5, 1838, 8, 1974, 15, 36, 119, 257, 85, 52, 486, 9, 6, 2, 8564, 63, 271, 6, 196, 96, 949, 4121, 4, 2, 7, 4, 2212, 2436, 819, 63, 47, 77, 7175, 180, 6, 227, 11, 94, 2494, 2, 13, 423, 4, 168, 7, 4, 22, 5, 89, 665, 71, 270, 56, 5, 13, 197, 12, 161, 5390, 99, 76, 23, 2, 7, 419, 665, 40, 91, 85, 108, 7, 4, 2084, 5, 4773, 81, 55, 52, 1901]
---label---
1


In [19]:
'''We can use the dictionary returned by imdb.get_word_index() to map the review back to the original words.'''
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print('---review with words---')
print([id2word.get(i, ' ') for i in X_train[1]])
print('---label---')
print(y_train[1])

---review with words---
['the', 'thought', 'solid', 'thought', 'senator', 'do', 'making', 'to', 'is', 'spot', 'nomination', 'assumed', 'while', 'he', 'of', 'jack', 'in', 'where', 'picked', 'as', 'getting', 'on', 'was', 'did', 'hands', 'fact', 'characters', 'to', 'always', 'life', 'thrillers', 'not', 'as', 'me', "can't", 'in', 'at', 'are', 'br', 'of', 'sure', 'your', 'way', 'of', 'little', 'it', 'strongly', 'random', 'to', 'view', 'of', 'love', 'it', 'so', 'principles', 'of', 'guy', 'it', 'used', 'producer', 'of', 'where', 'it', 'of', 'here', 'icon', 'film', 'of', 'outside', 'to', "don't", 'all', 'unique', 'some', 'like', 'of', 'direction', 'it', 'if', 'out', 'her', 'imagination', 'below', 'keep', 'of', 'queen', 'he', 'diverse', 'to', 'makes', 'this', 'stretch', 'and', 'of', 'solid', 'it', 'thought', 'begins', 'br', 'senator', 'and', 'budget', 'worthwhile', 'though', 'ok', 'and', 'awaiting', 'for', 'ever', 'better', 'were', 'and', 'diverse', 'for', 'budget', 'look', 'kicked', 'any', 'to

In [20]:
print(word2id)
print(id2word)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [21]:
#Maximum review length and minimum review length.
print('Maximum review length: {}'.format(
len(max((X_train + X_test), key=len))))

print('Minimum review length: {}'.format(
len(min((X_train + X_test), key=len))))

Maximum review length: 2697
Minimum review length: 70


In [23]:
'''In order to feed this data into our RNN, all input documents must have the same length. We will limit the maximum review length to maximum words by truncating 
longer reviews and padding shorter reviews with a null value (0). We can accomplish this task using the pad_sequences() function in Keras. Here, setting max_review_length 
to 500.'''

max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [25]:
X_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [ ]:
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

'''We first need to compile our model by specifying the loss function and optimizer we want to use while training, as well as any evaluation metrics 
we’d like to measure. Specify the appropriate parameters, including at least one metric ‘accuracy’.'''
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=100)

#Calculate Accuracy
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
#Now save the model in required directory
model.save('sentiment_analysis_final.h5')
print("Saved model to disk")

#Code to load the saved model
model = load_model('sentiment_analysis_final.h5')
print("Model Loaded")